In [1]:
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer, AutoModelForCausalLM
import json, os, shutil, re, random, io, requests, ctypes, sys, time, struct
from datasets import load_dataset,load_from_disk
import torch
import torch.nn as nn
import numpy as np
from tqdm import tqdm
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

/home/vipuser/miniconda3/envs/openr1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 06-02 16:59:34 [__init__.py:239] Automatically detected platform cuda.


2025-06-02 16:59:35,956	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
import pandas as pd
file_path = "../GRPO_data/val_prompt2.csv"
# model_path = "/home/LLM/models/Qwen2.5-0.5B-Instruct"
df = pd.read_csv(file_path)

In [3]:
model_path = "../SFT_model"
num_pre_Q = 1

In [4]:
sampling_params = SamplingParams(n=num_pre_Q, temperature=0.4, max_tokens=2048)
vllm_gen = LLM(model=model_path, gpu_memory_utilization=0.9)
tokenizer = AutoTokenizer.from_pretrained(model_path)

INFO 06-02 16:59:48 [config.py:2832] Downcasting torch.float32 to torch.float16.
INFO 06-02 16:59:54 [config.py:689] This model supports multiple tasks: {'score', 'classify', 'reward', 'embed', 'generate'}. Defaulting to 'generate'.
INFO 06-02 16:59:54 [config.py:1901] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 06-02 16:59:56 [core.py:61] Initializing a V1 LLM engine (v0.8.4) with config: model='../SFT_model', speculative_config=None, tokenizer='../SFT_model', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='auto', reasoning_backend=None), observability_config=Observab

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:00<00:00,  5.91it/s]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:00<00:00,  1.96it/s]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:00<00:00,  2.18it/s]



INFO 06-02 16:59:58 [loader.py:458] Loading weights took 1.00 seconds
INFO 06-02 16:59:59 [gpu_model_runner.py:1291] Model loading took 2.8876 GiB and 1.178282 seconds
INFO 06-02 17:00:06 [backends.py:416] Using cache directory: /root/.cache/vllm/torch_compile_cache/ffc93959dd/rank_0_0 for vLLM's torch.compile
INFO 06-02 17:00:06 [backends.py:426] Dynamo bytecode transform time: 7.34 s
INFO 06-02 17:00:09 [backends.py:132] Cache the graph of shape None for later use
INFO 06-02 17:00:29 [backends.py:144] Compiling a graph for general shape takes 21.68 s
INFO 06-02 17:00:39 [monitor.py:33] torch.compile takes 29.02 s in total
INFO 06-02 17:00:39 [kv_cache_utils.py:634] GPU KV cache size: 1,142,048 tokens
INFO 06-02 17:00:39 [kv_cache_utils.py:637] Maximum concurrency for 32,768 tokens per request: 34.85x
INFO 06-02 17:00:54 [gpu_model_runner.py:1626] Graph capturing finished in 15 secs, took 0.45 GiB
INFO 06-02 17:00:54 [core.py:163] init engine (profile, create kv cache, warmup model) t

In [5]:
system_prompt = """
You are a Schema linking assistant. Your task is to select appropriate table names and corresponding column names from the database in <database>...</database> based on the user's question in <question>...</question>. 
Follow this response template: <think>\nYour Thought Process (Why were these choices made?)\n</think>\n<answer>\n###Tables: (Related Table);\n###Columns: (Related Columns from Tables);\n</answer> 
"""
print(system_prompt)


You are a Schema linking assistant. Your task is to select appropriate table names and corresponding column names from the database in <database>...</database> based on the user's question in <question>...</question>. 
Follow this response template: <think>
Your Thought Process (Why were these choices made?)
</think>
<answer>
###Tables: (Related Table);
###Columns: (Related Columns from Tables);
</answer> 



In [6]:
def make_user_prompt(question, database_schema):
    return f"<question>{question}</question>\n<database>{database_schema}</database>"

In [7]:
def gen_answers(prompts):
    tip_text = []
    for x in prompts:
        tip_text.append(tokenizer.apply_chat_template([
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": x}], tokenize=False, add_generation_prompt=True))
    voutputs = vllm_gen.generate(tip_text, sampling_params, use_tqdm=False)
    answers = [];  ans_token_ids = []
    for v in voutputs:
        for z in v.outputs: 
            answers.append(z.text)
            ans_token_ids.append(z.token_ids)
    return answers, ans_token_ids

In [8]:
# from tqdm import tqdm
sample_list = []
correct_list = []
for index, row in tqdm(df.iterrows(), total=len(df)):
    correct_list.append(row["target_schema"])
    user_prompt = [make_user_prompt(row["question"],row["database_schema"])]
#     print(user_prompt)
#     print("<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<message_end>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
    response = gen_answers(user_prompt)[0][0]
    sample_list.append(response)
    if index % 100 ==0 :
        print(response)
        print(row["target_schema"])
        print("<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<next>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")

  0%|                                                                                                          | 1/1004 [00:02<34:56,  2.09s/it]

<think>
Okay, let's see. The user wants to know how many vocalists we have. So first, I need to figure out which tables and columns are relevant here.

Looking at the database schema, there are four tables: stadium, singer, concert, and singer_in_concert. The question is about vocalists, so the singer table is definitely a key here. The singer table has columns like Name, Country, Age, etc., which are probably relevant. 

The concert table links to the stadium via Stadium_ID, but since the question doesn't mention anything about concerts, maybe it's just about the singers themselves. The singer_in_concert table connects singers to concerts, but again, the question is about the singers' count, not the concerts. 

So the main table here is the singer table. The columns needed are Name, which would give the names of the vocalists. To count the number, we can use COUNT(distinct Name) or COUNT(Name) depending on whether we want unique names or all names. 

I need to check if there are any o

 10%|██████████▎                                                                                           | 101/1004 [05:37<1:07:05,  4.46s/it]

<think>
Okay, let's tackle this question: "What is the name of the different car manufacturers who produced a vehicle in 1970?" 

First, I need to figure out which tables and columns in the provided database schema are relevant. The question is about car manufacturers, their models, and the year 1970. So, the main tables here are likely the ones that store car information.

Looking at the database schema, there's a table called `car_names` which has columns like `Make`, `Model`, and `MakeId`. That seems directly related to the question. The `Make` column in `car_names` would hold the names of the car manufacturers. 

Next, the `cars_data` table has columns like `Year` which is 1970, and `MakeId` which links to `car_names` via `MakeId`. So, we can join `car_names` and `cars_data` on `MakeId` to get the year 1970 data. 

The other tables like `continents`, `countries`, `car_makers`, and `model_list` don't seem directly relevant here because the question isn't about continents, countries,

 20%|████████████████████▊                                                                                   | 201/1004 [10:39<25:48,  1.93s/it]

...

 90%|█████████████████████████████████████████████████████████████████████████████████████████████▎          | 901/1004 [48:15<06:31,  3.80s/it]

<think>
Okay, let's see. The user's question is asking for the people who own the most dogs. So, I need to figure out which tables and columns in the provided database would hold that information.

First, the key here is "people" and "dogs." The Owners table has columns like owner_id, first_name, last_name, etc. That makes sense because the Owners table likely stores details about the owners of the dogs. The Dogs table has owner_id as a foreign key, which links to the Owners table. So, each dog is associated with an owner via the owner_id.

Now, the question is about the number of dogs each owner has. That would involve counting the number of dogs per owner. The Dogs table has a column called dog_id, which is a foreign key referencing the Owners.owner_id. So, to get the count of dogs per owner, I need to count the number of dog_ids in the Dogs table that are linked to each owner_id in the Owners table.

Wait, but the answer mentions the tables Owners and Dogs, and the columns owner_id 

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████▋| 1001/1004 [53:27<00:07,  2.65s/it]

<think>
Okay, let's see. The user's question is asking, "How many available characteristic are there in total?" So, I need to figure out which tables and columns in the provided database schema would hold that information.

First, I'll look at the tables. There's a table called "Other_Available_Features" which has columns like feature_id, feature_type_code, feature_name, etc. That seems relevant because the question is about characteristics available, which likely refers to features or attributes of properties. The other tables like Ref_Feature_Types or Ref_Property_Types don't seem directly related to the question since they're about types of features and property types, not the actual characteristics themselves.

Now, checking the columns in Other_Available_Features. The feature_id column here would be the unique identifier for each available feature. The feature_type_code references Ref_Feature_Types, so that's probably the type of feature. The feature_name gives the name of the fea

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 1004/1004 [53:35<00:00,  3.20s/it]


In [9]:
print(len(sample_list))
print(len(correct_list))
print(correct_list[0])

1004
1004
###Tables: singer;
###Columns: singer.name, singer.song_release_year, singer.country, singer.singer_id, singer.song_name, singer.age;



In [10]:
import pandas as pd
write_dict = {}
write_dict["sample"] = sample_list
write_dict["correct"] = correct_list

csv_filename = 'GRPO_COLD.csv'
# 将字典转换为 DataFrame
df = pd.DataFrame(write_dict)
 
# 写入 CSV 文件
df.to_csv(csv_filename, index=False)
 
print(f"字典已成功写入到 {csv_filename}")

字典已成功写入到 GRPO_COLD.csv


In [11]:
def check_socore_label(ground_truth ,answer):

    # get ground truth      
    ground_truth_tables_match = re.search(r'Tables:\s*(.*?);', ground_truth)
    ground_truth_columns_match = re.search(r'Columns:\s*(.*?);', ground_truth)
    
    truth_tables = set(ground_truth_tables_match.group(1).split(', ')) if ground_truth_tables_match else set()
    truth_columns = set([col.strip() for col in ground_truth_columns_match.group(1).split(', ')]) if ground_truth_columns_match else set()
    
    truth_tables = {s.lower() for s in truth_tables}
    truth_columns = {s.lower() for s in truth_columns}
    
    tables_match =  re.search(r'Tables:\s*(.*?);', answer)
    columns_match = re.search(r'Columns:\s*(.*?);', answer)
    pre_tables = set(tables_match.group(1).split(', ')) if tables_match else set()
    pre_columns = set([col.strip() for col in columns_match.group(1).split(', ')]) if columns_match else set()
    
    pre_tables = {s.lower() for s in pre_tables}
    pre_columns = {s.lower() for s in pre_columns}

    table_format_len = len(truth_tables - (truth_tables - pre_tables))
    column_format_len = len(truth_columns - (truth_columns - pre_columns))
    
    
    table_format_acc =  table_format_len / len(truth_tables)
    column_format_acc =  column_format_len / len(truth_columns)
    
   
    return table_format_acc, column_format_acc

In [12]:
def strict_format_label(ground_truth ,answer):

    # get ground truth      
    ground_truth_tables_match = re.search(r'Tables:\s*(.*?);', ground_truth)
    ground_truth_columns_match = re.search(r'Columns:\s*(.*?);', ground_truth)
    
    truth_tables = set(ground_truth_tables_match.group(1).split(', ')) if ground_truth_tables_match else set()
    truth_columns = set([col.strip() for col in ground_truth_columns_match.group(1).split(', ')]) if ground_truth_columns_match else set()
    
    
    tables_match =  re.search(r'Tables:\s*(.*?);', answer)
    columns_match = re.search(r'Columns:\s*(.*?);', answer)
    pre_tables = set(tables_match.group(1).split(', ')) if tables_match else set()
    pre_columns = set([col.strip() for col in columns_match.group(1).split(', ')]) if columns_match else set()
    
    table_format = 0
    column_format = 0
    if pre_tables ==  truth_tables:
        table_format = 1
    
    if pre_columns ==  truth_columns:
        column_format = 1
   
    return table_format, column_format

In [13]:
table_acc_list = []
columns_acc_list = []
for index in range(len(sample_list)):
#     print(correct_list[index])
#     print(sample_list[index])
#     print("++++++++++++++++++++++++++++++++")
#     break
    table_format_reward, column_format_reward = check_socore_label(correct_list[index],sample_list[index])
    print("table_format_reward:",table_format_reward)
    print("column_format_reward:",column_format_reward)
    table_acc_list.append(table_format_reward)
    columns_acc_list.append(column_format_reward)
    print("+++++++++++++++++++++++++++")
#     gen_len_reward(sample_list[index])

table_format_reward: 1.0
column_format_reward: 0.16666666666666666
+++++++++++++++++++++++++++
table_format_reward: 1.0
column_format_reward: 0.16666666666666666
+++++++++++++++++++++++++++
table_format_reward: 1.0
column_format_reward: 1.0
+++++++++++++++++++++++++++
table_format_reward: 1.0
column_format_reward: 1.0
+++++++++++++++++++++++++++
table_format_reward: 1.0
column_format_reward: 0.5
+++++++++++++++++++++++++++
table_format_reward: 1.0
column_format_reward: 0.5
+++++++++++++++++++++++++++
table_format_reward: 1.0
column_format_reward: 0.3333333333333333
+++++++++++++++++++++++++++
table_format_reward: 1.0
column_format_reward: 0.6666666666666666
+++++++++++++++++++++++++++
table_format_reward: 1.0
column_format_reward: 1.0
+++++++++++++++++++++++++++
table_format_reward: 1.0
column_format_reward: 1.0
+++++++++++++++++++++++++++
table_format_reward: 0.0
column_format_reward: 0.0
+++++++++++++++++++++++++++
table_format_reward: 0.0
column_format_reward: 0.0
++++++++++++++++++

In [14]:
print("avg_table_format:",np.sum(table_acc_list)/len(table_acc_list))
print("avg_column_format:",np.sum(columns_acc_list)/len(columns_acc_list))

avg_table_format: 0.7906706507304117
avg_column_format: 0.5452084519066591


In [15]:
table_format_list = []
column_format_list = []
for index in range(len(sample_list)):
#     print(correct_list[index])
#     print(sample_list[index])
#     print("++++++++++++++++++++++++++++++++")
#     break
    table_format, column_format = strict_format_label(correct_list[index],sample_list[index])
    print("table_format:",table_format)
    print("column_format:",column_format)
    table_format_list.append(table_format)
    column_format_list.append(column_format)
    print("+++++++++++++++++++++++++++")
#     gen_len_reward(sample_list[index])

table_format: 1
column_format: 0
+++++++++++++++++++++++++++
table_format: 1
column_format: 0
+++++++++++++++++++++++++++
table_format: 1
column_format: 1
+++++++++++++++++++++++++++
table_format: 1
column_format: 1
+++++++++++++++++++++++++++
table_format: 1
column_format: 0
+++++++++++++++++++++++++++
table_format: 1
column_format: 0
+++++++++++++++++++++++++++
table_format: 1
column_format: 0
+++++++++++++++++++++++++++
table_format: 1
column_format: 0
+++++++++++++++++++++++++++
table_format: 1
column_format: 1
+++++++++++++++++++++++++++
table_format: 1
column_format: 1
+++++++++++++++++++++++++++
table_format: 0
column_format: 0
+++++++++++++++++++++++++++
table_format: 0
column_format: 0
+++++++++++++++++++++++++++
table_format: 0
column_format: 0
+++++++++++++++++++++++++++
table_format: 0
column_format: 0
+++++++++++++++++++++++++++
table_format: 1
column_format: 0
+++++++++++++++++++++++++++
table_format: 1
column_format: 0
+++++++++++++++++++++++++++
table_format: 1
column_f

In [16]:
print("avg_table_format:",np.sum(table_format_list)/len(table_format_list))
print("avg_column_format:",np.sum(column_format_list)/len(column_format_list))

avg_table_format: 0.5667330677290837
avg_column_format: 0.18127490039840638


In [17]:
# dts-bench

In [18]:
def get_Table_columns(ground_truth ,answer):
    ground_truth_tables_match = re.search(r'Tables:\s*(.*?);', ground_truth)
    ground_truth_columns_match = re.search(r'Columns:\s*(.*?);', ground_truth)

    truth_tables = set(ground_truth_tables_match.group(1).split(', ')) if ground_truth_tables_match else set()
    truth_columns = set([col.strip() for col in ground_truth_columns_match.group(1).split(', ')]) if ground_truth_columns_match else set()
    truth_tables = {s.lower() for s in truth_tables}
    truth_columns = {s.lower() for s in truth_columns}

    tables_match =  re.search(r'Tables:\s*(.*?);', answer)
    columns_match = re.search(r'Columns:\s*(.*?);', answer)
    pre_tables = set(tables_match.group(1).split(', ')) if tables_match else set()
    pre_columns = set([col.strip() for col in columns_match.group(1).split(', ')]) if columns_match else set()
    pre_tables = {s.lower() for s in pre_tables}
    pre_columns = {s.lower() for s in pre_columns}
    return truth_tables, truth_columns, pre_tables, pre_columns

In [19]:
total_samples = len(sample_list)

table_total_accuracy = 0
table_filtered_accuracy = 0
table_total_precision = 0
table_total_recall = 0


columns_total_accuracy = 0
columns_filtered_accuracy = 0
columns_total_precision = 0
columns_total_recall = 0


for index in range(len(sample_list)):
#     print(correct_list[index])
#     print(sample_list[index])
#     print("++++++++++++++++++++++++++++++++")
#     break
    precision = 0
    recall = 0
    truth_tables, truth_columns, pre_tables, pre_columns = get_Table_columns(correct_list[index],sample_list[index])
    if set(pre_tables) == set(truth_tables):
        table_total_accuracy += 1
    table_true_positives = len(set(pre_tables) & set(truth_tables))
    table_false_positives = len(set(pre_tables) - set(truth_tables))
    table_false_negatives = len(set(truth_tables) - set(pre_tables))
    
    if table_true_positives == len(truth_tables):
        table_filtered_accuracy += 1
    
    if len(pre_tables) > 0:
        precision = table_true_positives / (table_true_positives + table_false_positives)
        recall = table_true_positives / (table_true_positives + table_false_negatives)
    
    table_total_precision += precision
    table_total_recall += recall
    
    ########################cloumns####################################
    precision = 0
    recall = 0
    if set(pre_columns) == set(truth_columns):
        columns_total_accuracy += 1
    columns_true_positives = len(set(pre_columns) & set(truth_columns))
    columns_false_positives = len(set(pre_columns) - set(truth_columns))
    columns_false_negatives = len(set(truth_columns) - set(pre_columns))
    
    if columns_true_positives == len(truth_columns):
        columns_filtered_accuracy += 1
    
    if len(pre_columns) > 0:
        precision = columns_true_positives / (columns_true_positives + columns_false_positives)
        recall = columns_true_positives / (columns_true_positives + columns_false_negatives)
    
    columns_total_precision += precision
    columns_total_recall += recall

In [20]:
# Calculate average precision and recall
avg_precision = table_total_accuracy / total_samples
avg_recall = table_total_recall / total_samples

# Calculate total accuracy
accuracy = table_total_accuracy / total_samples
filtered_accuracy = table_filtered_accuracy / total_samples

print("Total Accuracy:", accuracy)
print("Filtered Accuracy:", filtered_accuracy)
print("Average Precision:", avg_precision)
print("Average Recall:", avg_recall)

Total Accuracy: 0.5667330677290837
Filtered Accuracy: 0.704183266932271
Average Precision: 0.5667330677290837
Average Recall: 0.7906706507304117


In [21]:
# Calculate average precision and recall
avg_precision = columns_total_accuracy / total_samples
avg_recall = columns_total_recall / total_samples

# Calculate total accuracy
accuracy = columns_total_accuracy / total_samples
filtered_accuracy = columns_filtered_accuracy / total_samples

print("Total Accuracy:", accuracy)
print("Filtered Accuracy:", filtered_accuracy)
print("Average Precision:", avg_precision)
print("Average Recall:", avg_recall)

Total Accuracy: 0.1942231075697211
Filtered Accuracy: 0.350597609561753
Average Precision: 0.1942231075697211
Average Recall: 0.5452084519066589
